# Step 4: Test and Compare Models

In this step, you'll test your trained model and compare it with EasyOCR's pre-trained model to evaluate performance.

## What You'll Learn:
- How to set up a custom OCR model for inference
- How to compare custom vs pre-trained models
- How to interpret OCR results and confidence scores

## 4.1 Setup: Copy Trained Model

First, copy your trained model to the `user_network_dir` directory where EasyOCR will load it.

**Important Files Needed:**
- `user_network_dir/custom.pth` - Your trained model weights
- `user_network_dir/custom.yaml` - Model configuration (architecture, character list)
- `user_network_dir/custom.py` - Model architecture definition

In [ ]:
# Create the user network directory if it doesn't exist
!mkdir -p ./user_network_dir

# Copy your trained model to the user network directory
# ⚠️ IMPORTANT: Change the date folder name to match your training run
# The folder name format is: TPS-VGG-BiLSTM-CTC-Seed1111-YYYYMMDD
!cp ./saved_models/TPS-VGG-BiLSTM-CTC-Seed1111-20251019/best_accuracy.pth ./user_network_dir/custom.pth

## 4.2 Prepare Test Images

Place your test images in the `demo_images` folder. These should be images similar to what your model will process in production (e.g., receipts, invoices, signs).

```bash
demo_images/
  ├── demo_image_01.jpg
  ├── demo_image_02.jpg
  └── ...
```

## 4.3 Test with Pre-trained Model (Baseline)

First, test with EasyOCR's pre-trained model to establish a baseline performance.

In [ ]:
from easyocr import Reader
import os

# Set GPU devices
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

# Initialize EasyOCR with pre-trained traditional Chinese model
print("Loading EasyOCR pre-trained model...")
reader_pretrained = Reader(['ch_tra'], gpu=True)

# Get test images
demo_images_dir = './demo_images'
test_files = [f for f in os.listdir(demo_images_dir) if not f.startswith('.')]
test_files.sort()

print(f"\n{'='*70}")
print("TESTING WITH EASYOCR PRE-TRAINED MODEL (Baseline)")
print(f"{'='*70}\n")

# Test each image - simplified version
for filename in test_files:
    file_path = os.path.join(demo_images_dir, filename)
    results = reader_pretrained.readtext(file_path)
    
    for bbox, text, confidence in results:
        print("filename: '%s', confidence: %.4f, string: '%s'" % (filename, confidence, text))

## 4.4 Test with Your Custom Model

Now test with your trained model. Make sure you have:
- ✅ `user_network_dir/custom.pth` (model weights)
- ✅ `user_network_dir/custom.yaml` (configuration)
- ✅ `user_network_dir/custom.py` (architecture)

In [ ]:
from easyocr import Reader
import os

# Set GPU devices
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

# Initialize EasyOCR with your custom model
print("Loading your custom trained model...")
reader_custom = Reader(
    ['ch_tra'], 
    gpu=True,
    model_storage_directory='./user_network_dir',
    user_network_directory='./user_network_dir',
    recog_network='custom'
)

# Get test images
demo_images_dir = './demo_images'
test_files = [f for f in os.listdir(demo_images_dir) if not f.startswith('.')]
test_files.sort()

print(f"\n{'='*70}")
print("TESTING WITH YOUR CUSTOM MODEL")
print(f"{'='*70}\n")

# Test each image - simplified version
for filename in test_files:
    file_path = os.path.join(demo_images_dir, filename)
    results = reader_custom.readtext(file_path)
    
    for bbox, text, confidence in results:
        print("filename: '%s', confidence: %.4f, string: '%s'" % (filename, confidence, text))

## 4.5 Side-by-Side Comparison

Compare both models on the same images to see which performs better.

In [8]:
from easyocr import Reader
import os

# Set GPU devices
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

# Initialize both models
print("Loading both models...")
reader_pretrained = Reader(['ch_tra'], gpu=True)
reader_custom = Reader(['ch_tra'], gpu=True,
                      model_storage_directory='./user_network_dir',
                      user_network_directory='./user_network_dir',
                      recog_network='custom')

# Get test images
demo_images_dir = './demo_images'
test_files = [f for f in os.listdir(demo_images_dir) if not f.startswith('.')]
test_files.sort()

print(f"\n{'='*80}")
print("SIDE-BY-SIDE COMPARISON: Pre-trained vs Custom Model")
print(f"{'='*80}\n")

# Compare results for each image
for filename in test_files:
    file_path = os.path.join(demo_images_dir, filename)
    print(f"📷 Testing: {filename}")
    print(f"{'-'*80}")
    
    # Test with pre-trained model
    results_pretrained = reader_pretrained.readtext(file_path)
    print("🔵 EasyOCR Pre-trained Model:")
    if results_pretrained:
        for bbox, text, confidence in results_pretrained:
            print("   confidence: %.4f, string: '%s'" % (confidence, text))
    else:
        print("   No text detected")
    
    print()
    
    # Test with custom model
    results_custom = reader_custom.readtext(file_path)
    print("🟢 Your Custom Model:")
    if results_custom:
        for bbox, text, confidence in results_custom:
            print("   confidence: %.4f, string: '%s'" % (confidence, text))
    else:
        print("   No text detected")
    
    print(f"\n{'='*80}\n")

Loading both models...

SIDE-BY-SIDE COMPARISON: Pre-trained vs Custom Model

📷 Testing: demo_image_01.jpg
--------------------------------------------------------------------------------
🔵 EasyOCR Pre-trained Model:
   confidence: 0.0146, string: '九龍青衣紐山路2號美景花園平壼72號舖'

🟢 Your Custom Model:
   confidence: 0.1838, string: '果'




/Users/wing199901/Downloads/smart-spend-ocr-workspace/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/wing199901/Downloads/smart-spend-ocr-workspace/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
